In [1]:
#%pip install graphframes

In [2]:
#run dependent notebooks
# %run ./etl_trusted_features.ipynb
%run ./read_file.ipynb
from graphframes import GraphFrame

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-64bcb3c2-3004-4229-ac1f-b7602fc41d05;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 129ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/19 23:54:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 23:54:28 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/19 23:54:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/19 23:54:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
#read in filtered data
df = readFirstEvents()

11:54PM UTC on Mar 19, 2023 --- read time: 16.14694881439209 seconds ---


In [4]:
#get a total count 
df.count()

948334

In [5]:
#reminder of shcema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- relationship: string (nullable = true)



In [6]:
#split into df_b and df_m for benign and malicious
df_m = df.where(col('malicious') == 1)
df_b = df.where(col('malicious') == 0)

In [7]:
#understand the day time spread of each
print('mal day: '+str(df_m.select('event_day').distinct().show()) + 
      'mal hour: '+str(df_m.select('event_hour').distinct().show()))
print('ben day: '+str(df_b.select('event_day').distinct().show()) + 
      'ben hour: '+str(df_b.select('event_hour').distinct().show()))

+---------+
|event_day|
+---------+
|       23|
+---------+

+----------+
|event_hour|
+----------+
|        19|
|        15|
|        17|
|        18|
|        16|
+----------+

mal day: Nonemal hour: None
+---------+
|event_day|
+---------+
|       23|
+---------+

+----------+
|event_hour|
+----------+
|        13|
|        16|
|        19|
|        15|
|        17|
|        14|
|        18|
+----------+

ben day: Noneben hour: None


In [8]:
#down select benign events to those that occured the same time at malicious events

df_b_ds = df_b.where((col('event_hour')<=19) & (col('event_hour')>=15))

In [9]:
#and count these events and malicious events
print("b: "+str(df_b_ds.count()))
print("m: "+str(df_m.count()))

b: 633914
m: 3624


In [10]:

# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_m.selectExpr('actorID as id').distinct()
dst_vertices = df_m.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_m.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')



In [11]:
src_vertices.limit(5).show()
dst_vertices.limit(5).show()
vertices.limit(5).show()
edges.limit(5).show()

+--------------------+
|                  id|
+--------------------+
|d5846376-9596-428...|
|4a3392b6-acc8-4c2...|
|64c0966e-66f8-452...|
|bff0865c-96aa-4fb...|
|96913629-c1c9-450...|
+--------------------+

+--------------------+
|                  id|
+--------------------+
|bd92be9f-1823-465...|
|3ddb5f5d-9a09-415...|
|ff43a0aa-4930-468...|
|db532b31-916a-428...|
|bb90307d-0998-403...|
+--------------------+

+--------------------+
|                  id|
+--------------------+
|d5846376-9596-428...|
|4a3392b6-acc8-4c2...|
|1006f812-b221-46f...|
|64c0966e-66f8-452...|
|bff0865c-96aa-4fb...|
+--------------------+

+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+--------------+-----------------+--------+--------------------+---------+--------+-----------------+---------------+---------+
|                 src|                 dst|           timestamp| object|action|            hostname|user_name|privileges|    ima

In [12]:
# Create GraphFrame
g = GraphFrame(vertices, edges)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
